In [1]:
import os

In [2]:
import tensorflow as tf
import numpy as np
import os
# import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import (Input, Conv2D, MaxPooling2D, Flatten, Dense,Dropout,RandomFlip, RandomRotation, Rescaling)
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [1]:
import tensorflow as tf
import os

In [3]:
from PIL import Image

In [5]:
img_data_skip=0
for folder_name in ('Cat','Dog'):
    folder_path=os.path.join('PetImages',folder_name)
    for f_name in os.listdir(folder_path):
        f_path=os.path.join(folder_path,f_name) 

        try:
            img_bytes = tf.io.read_file(f_path)
            tf.io.decode_image(img_bytes)
        except Exception as e:
          print(f"Bad file found and deleting on {f_path}")
          img_data_skip +=1
          os.remove(f_path)

print(f"Pictures deleted due to be corrupte {img_data_skip}")

Pictures deleted due to be corrupte 0


In [6]:


directory = "PetImages"


train_ds = tf.keras.utils.image_dataset_from_directory(
    directory,
    validation_split=0.3,
    subset="training",
    seed=1345,
    image_size=(180, 180),
    batch_size=10
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    directory,
    validation_split=0.3,
    subset="validation",
    seed=1345,
    image_size=(180, 180),
    batch_size=10
)


Found 24991 files belonging to 2 classes.
Using 17494 files for training.
Found 24991 files belonging to 2 classes.
Using 7497 files for validation.


In [26]:
model_sequencial=Sequential([
    Input(shape=(180,180,3)),
    RandomFlip('horizontal'),
    RandomRotation(0.3),
    Rescaling(1./255),
    Conv2D(32,2,activation='relu'),
    MaxPooling2D(),
    Conv2D(64,2,activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(100,activation='relu'),
    Dropout(0.3),
    Dense(50,activation='relu'),
    Dropout(0.3),
    Dense(1,activation='sigmoid')
])


model_sequencial.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model_sequencial.summary()



Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ random_flip_1 (RandomFlip)      │ (None, 180, 180, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_rotation_1               │ (None, 180, 180, 3)    │             0 │
│ (RandomRotation)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling_1 (Rescaling)         │ (None, 180, 180, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 179, 179, 32)   │           416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 89, 89, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 88, 88, 64)     │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 44, 44, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 123904)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 100)            │    12,390,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 50)             │         5,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,404,273 (47.32 MB)

 Trainable params: 12,404,273 (47.32 MB)

 Non-trainable params: 0 (0.00 B)

In [129]:
train_ds

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 180, 180, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [130]:
val_ds

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 180, 180, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [28]:
model_sequencial.fit(
    train_ds,
    epochs=10,
    validation_data=val_ds,
    callbacks=[EarlyStopping(monitor='val_accuracy',patience=3,restore_best_weights=True,mode='max'),
    ModelCheckpoint('best_models.keras',save_best_only=True,monitor='val_loss',mode='min')
    ]

)

Epoch 1/10
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 585s 334ms/step - accuracy: 0.5524 - loss: 0.6856 - val_accuracy: 0.6525 - val_loss: 0.6368
Epoch 2/10
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 616s 352ms/step - accuracy: 0.6364 - loss: 0.6518 - val_accuracy: 0.6496 - val_loss: 0.6366
Epoch 3/10
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 630s 360ms/step - accuracy: 0.6564 - loss: 0.6251 - val_accuracy: 0.6937 - val_loss: 0.5923
Epoch 4/10
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 601s 343ms/step - accuracy: 0.6687 - loss: 0.6093 - val_accuracy: 0.7036 - val_loss: 0.5721
Epoch 5/10
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 539s 308ms/step - accuracy: 0.6957 - loss: 0.5872 - val_accuracy: 0.7210 - val_loss: 0.5552
Epoch 6/10
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 606s 346ms/step - accuracy: 0.6999 - loss: 0.5756 - val_accuracy: 0.7334 - val_loss: 0.5359
Epoch 7/10
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 631s 360ms/step - accuracy: 0.7143 - loss: 0.5658 - val_accuracy: 0.7363 - val_loss: 0.5299
Epoch 8/10
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 640s 366ms/step - ac

In [16]:
model = tf.keras.models.load_model('best_model.keras')


In [24]:
img =  tf.keras.utils.load_img("cat3.jpg", target_size=(180, 180))
img_arrayw = tf.keras.utils.img_to_array(img)
img_batch = np.expand_dims(img_arrayw, axis=0)
# k=model_sequencial.predict(img_batch)


In [ ]:
if m[0][0] < 0.5:
    print(f"{m[0][0]*100 :.2f}% it's a cat")
else:
    print(f"{m[0][0]*100 :.2f}% is a dog")

In [25]:
# kelvin=best_model.keras
m=model.predict(img_batch)
print(m)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
[[0.8815858]]


In [ ]:
# class_names=['cat','dog']
# from 0.5 below cat
# cat3 incorrect
# cat inccorrect
# cat1 correct
# cat2 incorrect

# from 0.5 above  dog
# dog correctly predicted
# dog2 correctly predicted
# dog3 correctly predicted

In [ ]:
plt.figure(figsize=(8,9))
plt.plot()
plt.imshow(img)
plt.title(f"predciton :{'dog' if m[0][0] >= 0.5 else 'cat'}")
plt.axis('off')
plt.show()